In [1]:
import torch
import torch.nn as nn #network 函式
import torch.nn.functional as F #一些方法
import torch.optim as optim #優化方法
from torchvision import datasets, transforms #提供資料集 演算法
import matplotlib.pyplot as plt #圖表

In [ ]:
print(torch.__version__)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [3]:
##超參數
batch_size = 32 #批次大小
lr = 0.001 #學習率
num_epochs = 10 #代數

In [ ]:
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=None) #讀取資料集 放在 "./data"

images, labels = zip(*[(train_dataset[i][0], train_dataset[i][1]) for i in range(5)]) #資料分割 


fig, axes = plt.subplots(1, 5, figsize=(15, 3)) #視覺化資料
for ax, image, label in zip(axes, images, labels):
    ax.set_axis_off()
    ax.imshow(image, cmap='gray')
    ax.set_title(f'Label: {label}')
plt.show()

In [5]:
transform = transforms.Compose([
    transforms.ToTensor(),  # shape H，W，C —> C，H，W
    transforms.Resize((28,28)),
    transforms.Normalize((0.1307,), (0.3081,)) # 將資料壓縮至一個範圍，例如：-1,1，這樣跑的效率會比較高
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform)


train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)



In [6]:
class DNN(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28*28, 400) #第一層全連接層 輸入28*28 輸出400 
        self.fc2 = nn.Linear(400, 200) #第二層全連接層 輸入400 輸出200 
        self.fc3 = nn.Linear(200, 10) #第二層全連接層 輸入400 輸出200 
        
    def forward(self, x): #
        x = x.view(-1, 28*28) #將圖像攤平  28*28:有28*28個像素
        x = F.sigmoid(self.fc1(x)) #將x丟進fc1後 應用sigmoid激活函數
        x = F.sigmoid(self.fc2(x)) #將x丟進fc2後 應用sigmoid激活函數
        x = self.fc3(x) #將x丟進fc3
        
        return F.softmax(x, dim=1) #輸出x經過softmax後的結果
    
model = DNN()
model = model.to(device)

## 優化器
# optimizer = torch.optim.SGD(model.parameters(), lr=LR)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
# optimizer = torch.optim.AdamW(model.parameters(), lr=LR)
# optimizer = torch.optim.Adagrad(model.parameters(), lr=LR)

# criterion = nn.MSELoss().to(device)
# criterion = nn.L1Loss().to(device)
# criterion = nn.BCELoss().to(device)
criterion = nn.CrossEntropyLoss().to(device)  #交叉商

In [ ]:
def accuracy(pred: torch.Tensor, label: torch.Tensor):
  _, pred_label = pred.max(1)
  num_correct = (pred_label == label).sum().item()
  acc = num_correct / label.shape[0]
  return acc


metric = {'loss': [], 'acc': []}
for i_epoch in range(num_epochs):
  train_loss = [] #宣告train的loss
  train_acc = [] #宣告train的acc
  model.train(mode=True) #模型轉為訓練模式
  for i_batch, (image, label) in enumerate(train_loader): #迭代每個batch 
    image = image.to(device) #圖片丟進gpu
    label = label.to(device) #label丟進gpu

    pred = model.forward(image) #預測pred 將圖片丟進模型
    loss = criterion(pred, label)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    train_loss += [loss.item()]
    train_acc += [accuracy(pred, label)]
  metric['loss'] += [sum(train_loss)/ len(train_loader)]
  metric['acc'] += [sum(train_acc)/ len(train_loader)]
  print(f'Epoch[{i_epoch+1}/{num_epochs}] loss: {metric["loss"][-1]}, acc: {metric["acc"][-1]}')

In [ ]:
plt.figure(figsize=(10, 5)) #新增10*5畫面
plt.plot(range(1,num_epochs+1),metric['loss'], label='Training Loss') #點圖 
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss Over Time')
plt.legend() #線條
plt.show()

In [ ]:
model.eval()
total_correct = 0
total_samples = 0

with torch.no_grad():
    for image, label in test_loader:
      image = image.to(device)
      label = label.to(device)
      pred = model.forward(image)
      _, pred_label = pred.max(1)
      total_correct += (pred_label == label).sum().item()
      total_samples += label.size(0)


print(f'Total correct: {total_correct}')
print(f'Total samples: {total_samples}')
test_acc = total_correct / total_samples
print(f'Test Accuracy: {test_acc}')

In [ ]:
from pathlib import Path
from PIL import Image
import torch


image = Image.open(r'/content/0.png')


transform = transforms.Compose([
    transforms.Lambda(lambda x: x.convert('RGB')),
    transforms.ToTensor(),  # shape H，W，C —> C，H，W
    transforms.Resize((28,28)),
    transforms.Grayscale(),
    transforms.Lambda(lambda x: 1.0 - x)
    #transforms.Normalize((0.1307,), (0.3081,)) # -1,1
])

transformed_image = transform(image).unsqueeze(0)

model.eval()
with torch.no_grad():

    outputs = model(transformed_image.to(device))
    print(outputs)

    _, predicted = torch.max(outputs, 1)
    print(predicted)


image

In [ ]:
from pathlib import Path
model_path = Path('./models/model_dnn.pt')
model_path.parent.mkdir(parents=True, exist_ok=True)
torch.jit.script(model).save(model_path)
